# Projet Info804

_Raphaëlle Huynh / Samy Amarouche_

## 1. Introduction et fondements théoriques

- Explication des concepts fondamentaux de représentation vectorielle des mots
- Histoire et évolution des embeddings de mots
- Principes mathématiques sous-jacents (calcul de probabilités, descente de gradient, etc.)

!!! A Faire

### 2. Préparation des données

- Création d'un pipeline de prétraitement textuel
  - Normalisation (mise en minuscule, suppression des caractères spéciaux)
  - Tokenisation
  - Gestion des mots rares (filtrage par fréquence)
- Construction du vocabulaire et mapping mot-indice
- Génération des paires d'apprentissage (contexte/cible)
- Implémentation de techniques d'échantillonnage négatif et de sous-échantillonnage de mots fréquents

In [1]:
import numpy as np
from src import TextPreprocessor
from src import CBOWModel
from src import SkipGramModel
from src import generate_skipgram_data
from src import generate_cbow_data
from src import plot_embeddings
from src import ANNClassifier
from src import generate_negative_samples
from src import subsample_frequent_words

#### Importation de la base de donnée

In [2]:
import pandas as pd

def load_imdb_data(filepath):
    """
    Charge le dataset IMDB et retourne les critiques textuelles.
    """
    data = pd.read_csv(filepath)
    reviews = data['review'].tolist()
    return reviews

# Exemple d'utilisation
filepath = "data/IMDB Dataset.csv"
corpus = load_imdb_data(filepath)
print(f"Nombre de phrases : {len(corpus)}")

Nombre de phrases : 50000


### Création de la pipeline de prétraitement et construction du vocabulaire et mapping mot-indice et Implémentation de techniques d'échantillonnage négatif et de sous-échantillonnage de mots fréquents

In [3]:
tp = TextPreprocessor(min_freq=5)

corpus = " ".join(corpus)

tokens = tp.preprocess(corpus)
tp.build_vocab(tokens)
encoded = tp.encode(tokens)

# Sous-échantillonnage des mots fréquents
subsampled_tokens = subsample_frequent_words(tokens, threshold=1e-5)
print(f"Nombre de tokens après sous-échantillonnage : {len(subsampled_tokens)}")

Nombre de tokens après sous-échantillonnage : 1611509


### Génération des paires d'apprentissage (contexte/cible)

In [4]:
# Générer les données pour CBOW
X, y = generate_cbow_data(encoded, window=2)

### Implémentation du modèle CBOW (Continuous Bag of Words)

In [ ]:
cbow = CBOWModel(
    vocab_size=len(tp.word2idx),
    embedding_dim=10,
    context_size=4,
    use_ann=True,
    optimizer='adam',
    learning_rate=0.01
)

: 

In [ ]:
cbow.summary()
cbow.train(X, y, epochs=100)
cbow_embeddings = cbow.get_embeddings()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 4)              │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embedding (Embedding)           │ (None, 4, 10)          │       447,410 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lambda (Lambda)                 │ (None, 10)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 44741)          │       492,151 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 939,561 (3.58 MB)

 Trainable params: 939,561 (3.58 MB)

 Non-trainable params: 0 (0.00 B)